In [1]:
%load_ext watermark
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from myst_nb import glue
from IPython.display import Markdown as md
from slugify import slugify

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

import plastockconf as psc
from plastockconf import name_zones, name_particles, name_frequentation, name_situation
from plastockconf import particle_groups, name_substrate, name_distance, table_css_styles, table_css_styles_top

from plastock import attribute_summary, attribute_summary_test, attribute_summary_grid, add_table_to_page
import plastock as pstk
import reportclass as rc

def translate_describe(x, value_column, transpose: bool = False):
    described = x.to_dict()
    described.pop("count")
    described["moyenne"] = described.pop("mean")
    described["écart-type"] = described.pop("std")
    df = pd.DataFrame(described.items())
    df.set_index(0, inplace=True)
    df.rename(columns={1:value_column}, inplace=True)
    df.index.name = None
    
    if transpose:
        df = df.T
        
    return df
import reportclass as rc
import plastock as pst
language_maps = rc.language_maps()

glue('blank_caption', " ", display=False)

section = 'MP'
page = ""

work_data = pd.read_csv("data/end_pipe/long_form_micro.csv")# thes samples were not completed
beach_data = pd.read_csv("data/end_pipe/asl_beaches.csv")

# the regional labels for each survey location
regions = pd.read_csv("data/end_pipe/lac_leman_regions.csv")
regions.set_index('slug', drop=True, inplace=True)

# the city name of the survey locations
city_map = pd.read_csv('data/end_pipe/city_map.csv')
city_map.set_index('slug', inplace=True)

In [2]:
# ! combine souples et dur !
fibers = work_data.loc[work_data.objet == 'fibres'].copy()
not_fibers = work_data.loc[work_data.objet != 'fibres'].copy()

not_fibers['objet'] = 'fragments'

work_datai = pd.concat([fibers, not_fibers])

In [3]:
def name_the_new_distance(x, less='<= 500 m', more = '> 500 m'):
    if x == 1:
        return less
    else:
        return more

def name_the_new_freq(x, new):
    if x <= 2:
        return new
    else:
        return 'Elévée'


# the feature variables are added to the work_data
ti = work_datai.copy()
features = ['frequentation', 'situation', 'orientation', 'distance']

work_datai['slug'] = work_datai.Plage.apply(lambda x: slugify(x))
work_datai['region'] = work_datai.slug.apply(lambda x: regions.loc[x, 'alabel'])
work_datai['city'] =  work_datai.slug.apply(lambda x: city_map.loc[x, 'city'])

work_data['particules'] = work_data.compte
work_datai["particules"] = work_datai.compte

work_data['slug'] = work_data.Plage.apply(lambda x: slugify(x))
work_data['region'] = work_data.slug.apply(lambda x: regions.loc[x, 'alabel'])
work_data['city'] =  work_data.slug.apply(lambda x: city_map.loc[x, 'city'])


In [4]:
# combining variables
env_plastock = work_datai.copy()


# the substrat and distance features are being combined
# the two lowest and the two highest of each group are being combined
# substrat is a matter of combining different granularities. They are being grouped as
# sand and gravel.
# distance is now grouped by locations either less than or equal to 500 meters
env_plastock.loc[env_plastock.substrat <= 2, 'substrat'] = 1
env_plastock.loc[env_plastock.substrat > 2, 'substrat'] = 2
env_plastock.loc[env_plastock.distance <= 2, 'distance'] = 1
env_plastock.loc[env_plastock.distance > 2, 'distance'] = 2
env_plastock.loc[env_plastock["fréquentation"] <= 2, 'fréquentation'] = 2
work_data_combined = env_plastock.copy()

In [5]:
# removing the sample values greater than the 99th percentile
wk_dt =work_datai.groupby('échantillon', as_index=False).compte.sum()
limit = np.quantile(wk_dt.compte.values, .99)

test = wk_dt[wk_dt.compte == 0]["échantillon"].unique()
not_these = wk_dt[wk_dt.compte > limit +1]["échantillon"].unique()
not_these = [*not_these, *test]


wd_10 = work_datai[~work_datai["échantillon"].isin(not_these)].copy()
wd_10dt = wd_10.groupby('échantillon', as_index=False).compte.sum()

test = wk_dt[wk_dt.compte == 0]["échantillon"].unique()

In [6]:
wk_dt =wd_10.groupby('échantillon', as_index=False).compte.sum()

# table MP-1
sit_disp = pd.DataFrame(translate_describe(wk_dt.compte.describe(), "résultats"))
sit_disp.loc["échantillon", "résultats"] = wd_10["échantillon"].nunique()
sit_disp.loc["total", "résultats"] = wd_10.compte.sum()
sit_disp["résultats"] = sit_disp["résultats"].astype(int)
sit_disp = sit_disp.style.set_table_styles(table_css_styles).format(**psc.format_kwargs)
sit_disp = add_table_to_page(sit_disp, 1, "", section, page, "")
glue('sit_disp_micro', sit_disp, display=False)

In [7]:
cgp_c = beach_data[["Plage", "x", "y"]]
cgp_c.set_index("Plage", inplace=True, drop=True)

map_data = wk_dt.copy()
plage_map = work_data[["Plage", "échantillon"]].copy()
plage_map = plage_map.drop_duplicates(["Plage", "échantillon"])
plage_map.set_index("échantillon", drop=True, inplace=True)
map_data["Plage"] = map_data["échantillon"].apply(lambda x: plage_map.loc[x, "Plage"])

map_data = map_data.groupby("Plage", as_index=False).compte.mean()
map_data["compte"] = map_data.compte.astype(int)
map_data["lat"] = map_data.Plage.apply(lambda x: cgp_c.loc[x, "x"])
map_data["lon"] = map_data.Plage.apply(lambda x: cgp_c.loc[x, "y"])

map_data.to_csv("resources/maps/micro_count_only.csv", index=False)

In [8]:
# figure 1.2
fig, axs = plt.subplots(2, 2, figsize=(8,8))

wk_dt = wd_10.groupby('échantillon', as_index=False).particules.sum()
vals = 'particules'
object_column = vals

ylim = np.quantile(wk_dt.particules.values, .99)
xlim = np.quantile(wk_dt.particules.values, .99)

sns.scatterplot(wk_dt, x="échantillon", y=vals, label='Pla\'stock micros particules', ax=axs[0, 0])
sns.boxplot(wk_dt, y=vals,  showfliers=True, ax=axs[0, 1], dodge=False)
sns.histplot(wk_dt, x=vals,  ax=axs[1, 0], stat='probability', kde=True)
sns.ecdfplot(wk_dt, x=vals,  ax=axs[1, 1])

axs[0, 0].set_ylim(-1, ylim)
axs[0, 1].set_ylim(-1, ylim)
axs[1, 1].set_xlim(-1, xlim)
axs[1, 0].set_xlim(-1, xlim)
# Hide X and Y axes tick marks
axs[0,0].set_xticks([])


axs[0, 0].set_xlabel("échantillon")
axs[0, 0].set_ylabel("Particules")

axs[1, 0].set_xlabel("Particules")
axs[1, 0].set_ylabel("Probabilité")
axs[0, 1].set_xlabel("")
axs[0, 1].set_ylabel("Particules")
axs[0, 1].tick_params(axis="x", rotation=45)

axs[1, 1].set_xlabel("Particules")
axs[0,0].set_title("Total par échantillon", loc="left")
axs[0,1].set_title("Boîte de Tukey", loc="left")
axs[1,0].set_title("Histogramme", loc="left")
axs[1,1].set_title("Fonction de répartition", loc="left")
plt.subplots_adjust(wspace=.3)


plt.tight_layout()

glue('micro-situataion-sa', fig, display=False)
plt.close()

In [9]:
# composition of total
# work_data has the excluded samples
parts = work_data.groupby(["objet"], as_index=False).compte.sum()
parts["% du total"] = parts.compte/parts.compte.sum()

In [10]:
# table mp2
data = work_data[~work_data["échantillon"].isin(not_these)].groupby(['échantillon', 'objet'], as_index=False).particules.sum()
table2 = data.groupby('objet').particules.describe()
table2 = table2.reindex(['fibres', 'fdure', 'souple'])
table2 = table2.astype('int')
language_maps = rc.language_maps()
g = language_maps['fr']
new_names = {x:g.loc[x, 'fr'] for x in table2.columns if x in g.index}


table2['Particule'] = table2.index.map(lambda x: psc.particle_groups[x])
table2.rename(columns=new_names, inplace=True)
caption = "Résultats par forme d'objet."

table2 = table2[[table2.columns[-1], *table2.columns[:-1]]].style.set_table_styles(table_css_styles).hide(axis=0)
table2 = pst.add_table_to_page(table2, 2, caption, section, page, "", format_index="both")
glue('table2', table2, display=False)

In [11]:
# table mp3
data = wd_10.groupby(['échantillon', 'objet'], as_index=False).particules.sum()
table3 = data.groupby('objet').particules.describe()
table3 = table3.astype('int')
table3['Particule'] = [x.capitalize() for x in table3.index]
table3.rename(columns=new_names, inplace=True)
table3 = table3[[table3.columns[-1], *table3.columns[:-1]]].style.set_table_styles(table_css_styles)

caption = "Résultats par forme d'objet, fragments durs et souples agrégée."
table3 = pst.add_table_to_page(table3, 3, caption, section, page, "", format_index='both')
table3 = table3.hide(axis='index')
glue('table3', table3, display=False)

In [12]:
# mp4
obj = 'fibres'
feature = 'substrat'

cols = ['échantillon', feature,  'objet']

data = work_data[~work_data["échantillon"].isin(not_these)].copy()
data = data[data.objet == obj].groupby(cols, as_index=False).particules.sum()
table4 = data.groupby(feature).particules.describe()
table4 = table4.astype('int')
table4[feature] = table4.index.map(lambda x: name_substrate[x])
table4.set_index(feature, inplace=True, drop=True)
table4.index.name = None
table4.rename(columns=new_names, inplace=True)



table4 = table4.style.set_table_styles(table_css_styles)
caption = "Fibres, résultats par subbstrat."


table4 = add_table_to_page(table4, 4, caption, section, page, "", format_index='both')
glue('table4', table4, display=False)

In [13]:
# mp5
obj = 'fibres'
feature = 'fréquentation'

cols = ['échantillon', feature,  'objet']

data = work_data[~work_data["échantillon"].isin(not_these)].copy()
data = data[data.objet == obj].groupby(cols, as_index=False).particules.sum()
table4 = data.groupby(feature).particules.describe()
table4 = table4.astype('int')
table4[feature] = table4.index.map(lambda x: name_frequentation[x])
table4.set_index(feature, inplace=True, drop=True)
table4.index.name = None
table4.rename(columns=new_names, inplace=True)



table4 = table4.style.set_table_styles(table_css_styles)
caption = "Fibres, résultats par fréquentation"


table5 = add_table_to_page(table4, 5, caption, section, page, "", format_index='both')
glue('table5', table5, display=False)

In [14]:
# mp6
obj = 'fibres'
feature = 'situation'

cols = ['échantillon', feature,  'objet']

data = work_data[~work_data["échantillon"].isin(not_these)].copy()
data = data[data.objet == obj].groupby(cols, as_index=False).particules.sum()
table4 = data.groupby(feature).particules.describe()
table4 = table4.astype('int')
table4[feature] = table4.index.map(lambda x: name_situation[x])
table4.set_index(feature, inplace=True, drop=True)
table4.index.name = None
table4.rename(columns=new_names, inplace=True)



table4 = table4.style.set_table_styles(table_css_styles)
caption = "Fibres, résultats par situation"


table6 = add_table_to_page(table4, 6, caption, section, page, "", format_index='both')
glue('table6', table6, display=False)

In [15]:
# mp7
obj = 'fibres'
feature = 'distance'

cols = ['échantillon', feature,  'objet']

data = work_data[~work_data["échantillon"].isin(not_these)].copy()
data = data[data.objet == obj].groupby(cols, as_index=False).particules.sum()
table4 = data.groupby(feature).particules.describe()
table4 = table4.astype('int')
table4[feature] = table4.index.map(lambda x: name_distance[x])
table4.set_index(feature, inplace=True, drop=True)
table4.index.name = None
table4.rename(columns=new_names, inplace=True)



table4 = table4.style.set_table_styles(table_css_styles)
caption = "Fibres, résultats par distance du parking"


table7 = add_table_to_page(table4, 7, caption, section, page, "", format_index='both')
glue('table7', table7, display=False)

In [16]:
#mp8
obj = 'fdure'
feature = 'substrat'

cols = ['échantillon', feature,  'objet']

data = work_data[~work_data["échantillon"].isin(not_these)].copy()
data = data[data.objet == obj].groupby(cols, as_index=False).particules.sum()
table4 = data.groupby(feature).particules.describe()
table4 = table4.astype('int')
table4[feature] = table4.index.map(lambda x: name_substrate[x])
table4.set_index(feature, inplace=True, drop=True)
table4.index.name = None
table4.rename(columns=new_names, inplace=True)



table4 = table4.style.set_table_styles(table_css_styles)
caption = "Fragments durs, résultats par substrat"


table4 = add_table_to_page(table4, 8, caption, section, page, "", format_index='both')
glue('table8', table4, display=False)

In [17]:
# mp9
obj = 'fdure'
feature = 'fréquentation'

cols = ['échantillon', feature,  'objet']

data = work_data[~work_data["échantillon"].isin(not_these)].copy()
data = data[data.objet == obj].groupby(cols, as_index=False).particules.sum()
table4 = data.groupby(feature).particules.describe()
table4 = table4.astype('int')
table4[feature] = table4.index.map(lambda x: name_frequentation[x])
table4.set_index(feature, inplace=True, drop=True)
table4.index.name = None
table4.rename(columns=new_names, inplace=True)



table4 = table4.style.set_table_styles(table_css_styles)
caption =  "Fragments durs, résultats par fréquentation"


table5 = add_table_to_page(table4, 9, caption, section, page, "", format_index='both')
glue('table9', table5, display=False)

In [18]:
# mp10
obj = 'fdure'
feature = 'situation'

cols = ['échantillon', feature,  'objet']

data = work_data[~work_data["échantillon"].isin(not_these)].copy()
data = data[data.objet == obj].groupby(cols, as_index=False).particules.sum()
table4 = data.groupby(feature).particules.describe()
table4 = table4.astype('int')
table4[feature] = table4.index.map(lambda x: name_situation[x])
table4.set_index(feature, inplace=True, drop=True)
table4.index.name = None
table4.rename(columns=new_names, inplace=True)



table4 = table4.style.set_table_styles(table_css_styles)
caption =  "Fragments durs, résultats par situtation"


table6 = add_table_to_page(table4, 10, caption, section, page, "", format_index='both')
glue('table10', table6, display=False)

In [19]:
# mp11
obj = 'fdure'
feature = 'distance'

cols = ['échantillon', feature,  'objet']

data = work_data[~work_data["échantillon"].isin(not_these)].copy()
data = data[data.objet == obj].groupby(cols, as_index=False).particules.sum()
table4 = data.groupby(feature).particules.describe()
table4 = table4.astype('int')
table4[feature] = table4.index.map(lambda x: name_distance[x])
table4.set_index(feature, inplace=True, drop=True)
table4.index.name = None
table4.rename(columns=new_names, inplace=True)



table4 = table4.style.set_table_styles(table_css_styles)
caption =  "Fragments durs, résultats par distance du parking"


table7 = add_table_to_page(table4, 11, caption, section, page, "", format_index='both')
glue('table11', table7, display=False)

In [20]:
# mp12
obj = 'souple'
feature = 'substrat'

cols = ['échantillon', feature,  'objet']

data = work_data[~work_data["échantillon"].isin(not_these)].copy()
data = data[data.objet == obj].groupby(cols, as_index=False).particules.sum()
table4 = data.groupby(feature).particules.describe()
table4 = table4.astype('int')
table4[feature] = table4.index.map(lambda x: name_substrate[x])
table4.set_index(feature, inplace=True, drop=True)
table4.index.name = None
table4.rename(columns=new_names, inplace=True)



table4 = table4.style.set_table_styles(table_css_styles)
caption = "Fragments souples, résultats par substrat"


table4 = add_table_to_page(table4, 12, caption, section, page, "", format_index='both')
glue('table12', table4, display=False)

In [21]:
# mp13
obj = 'souple'
feature = 'fréquentation'

cols = ['échantillon', feature,  'objet']

data = work_data[~work_data["échantillon"].isin(not_these)].copy()
data = data[data.objet == obj].groupby(cols, as_index=False).particules.sum()
table4 = data.groupby(feature).particules.describe()
table4 = table4.astype('int')
table4[feature] = table4.index.map(lambda x: name_frequentation[x])
table4.set_index(feature, inplace=True, drop=True)
table4.index.name = None
table4.rename(columns=new_names, inplace=True)



table4 = table4.style.set_table_styles(table_css_styles)
caption = "Fragments souples, résultats par fréquentation"


table5 = add_table_to_page(table4, 13, caption, section, page, "", format_index='both')
glue('table13', table5, display=False)

In [22]:
# mp 14
obj = 'souple'
feature = 'situation'

cols = ['échantillon', feature,  'objet']

data = work_data[~work_data["échantillon"].isin(not_these)].copy()
data = data[data.objet == obj].groupby(cols, as_index=False).particules.sum()
table4 = data.groupby(feature).particules.describe()
table4 = table4.astype('int')
table4[feature] = table4.index.map(lambda x: name_situation[x])
table4.set_index(feature, inplace=True, drop=True)
table4.index.name = None
table4.rename(columns=new_names, inplace=True)



table4 = table4.style.set_table_styles(table_css_styles)
caption = "Fragments souples, résultats par situation"


table6 = add_table_to_page(table4, 14, caption, section, page, "", format_index='both')
glue('table14', table6, display=False)

In [23]:
# mp15
obj = 'souple'
feature = 'distance'

cols = ['échantillon', feature,  'objet']

data = work_data[~work_data["échantillon"].isin(not_these)].copy()
data = data[data.objet == obj].groupby(cols, as_index=False).particules.sum()
table4 = data.groupby(feature).particules.describe()
table4 = table4.astype('int')
table4[feature] = table4.index.map(lambda x: name_distance[x])
table4.set_index(feature, inplace=True, drop=True)
table4.index.name = None
table4.rename(columns=new_names, inplace=True)



table4 = table4.style.set_table_styles(table_css_styles)
caption = "Fragments souples, résultats par distance du parking"


table7 = add_table_to_page(table4, 15, caption, section, page, "", format_index='both')
glue('table15', table7, display=False)

In [24]:
# mp16
bn = work_data.groupby(["region", "objet"], as_index=False).particules.mean()
bn = bn.pivot(columns="region", index="objet")
bn.index.name = None
bn.columns.name =None
bn = bn.droplevel(0, axis=1)
bn.columns.name = None
bn["object"] = bn.index.map(lambda x: particle_groups[x])
bn.set_index('object', inplace=True, drop=True)
bn.index.name = None
bn = bn.astype('int')
bn = bn.reindex(["Fibre", "Particule rigide", "Particule souple"])
bn.loc["Total"] = bn.sum()

bn = bn.style.set_table_styles(table_css_styles)
caption = "Résultats moyens de l'échantillon par région et par forme"


tablei = add_table_to_page(bn, 16, caption, section, page, "", format_index='both')
glue('table16', tablei, display=False)


In [25]:
# mp17
bn = work_data.groupby(["region", "objet"], as_index=False).particules.sum()
bn = bn.pivot(columns="region", index="objet")
bn.index.name = None
bn.columns.name =None
bn = bn.droplevel(0, axis=1)
bn.columns.name = None
bn["object"] = bn.index.map(lambda x: particle_groups[x])
bn.set_index('object', inplace=True, drop=True)
bn.index.name = None
bn = bn.astype('int')
bn = bn.reindex(["Fibre", "Particule rigide", "Particule souple"])
bn.loc["Total"] = bn.sum()

bn = bn.style.set_table_styles(table_css_styles)
caption = "Nombre total de particules collectées par région et par forme"


tablei = add_table_to_page(bn, 17, caption, section, page, "", format_index='both')
glue('table17', tablei, display=False)

In [26]:
# mp18
nsamples = work_data.groupby(["region"], as_index=False)["échantillon"].nunique()
nsamples.set_index("region", inplace=True, drop=True)
nsamples.rename(columns={"échantillon": "Echantillons"}, inplace=True)
nsamples.index.name = None
nsamples = nsamples.style.set_table_styles(table_css_styles)
caption = "Nombre d'échantillons"


tablei = add_table_to_page(nsamples, 18, caption, section, page, "", format_index='both')
glue('table18', tablei, display=False)

In [27]:
#
position_totals = wd_10.groupby(["Plage","échantillon", "position"], as_index=False).particules.sum()
position_totals["position"] = position_totals.position.apply(lambda x: name_zones[x])

fig, ax = plt.subplots(1, figsize=(9,5))

sns.boxplot(data=position_totals, x="Plage", y="particules", hue="position", ax=ax, showfliers=False)


ax.set_xlabel("")
ax.set_ylabel("Particules")
ax.tick_params(axis="x", labelrotation=90)
# capitalize_x_and_y_axis_labels(ax)
# capitalize_legend_components(ax)

plt.tight_layout()
glue('fig-mp3', fig, display=False)

plt.close()

# Micros particules 

__Format__ 

Le format suit celui de l'annexe pour les [microplastiques](micro_atts) . Nous incluons également l'analyse utilisant des variables combinées selon la méthode décrite dans la section [Résultats précédents](previous_results).

:::{card}
:margin: 3

```{image} resources/maps/chapter_one_map.jpeg
:alt: chapter one map
:align: center
:name: chapter_one_map
```
+++
<b>Fig 1.1 :</b> Carte des résultats de l'étude. Nombre moyenne de particules. Le diamètre du marqueur est relatif à la compte moyenne de particules,
:::

__Le système de mesure.__

Dans cette section, les unités sont le nombre de particules par conteneur. Le récipient a des côtés de 10 cm X 10 cm X 5 cm ou 0,0005 m³.

## Résultats

::::{grid} 1 1 2 2

:::{grid-item}

{glue}`sit_disp_micro`

:::

:::{grid-item}

D'après le tableau [A1-4](micro-table-A1-4), le nombre maximum de particules était de 2 991. Ce chiffre a été enregistré à la Pichette, où 2 593 fibres ont été comptées dans un échantillon. Le 99e percentile de tous les totaux d'échantillons est de 1 060 particules par échantillon. Ce chiffre a été dépassé trois fois, deux fois à Port Choiseul et une fois à la Pichette.

Nous considérons ici les données limitées au 99e percentile, c'est-à-dire que nous éliminons les trois échantillons qui ont dépassé 1 061 particules.

<b>Table MP-1 :</b> Distribution des résultats de l’ensemble de l’échantillons Pla'stock. Toutes les valeurs inférieures à 1 061 (99e percentile).
:::

::::

:::{card} 

{glue}`micro-situataion-sa`

+++
<b> Figure 1.2 :</b> Pla'stock 2022: distribution des résultats de l’ensemble de l’échantillons Pla'stock. L'élimination de ces trois échantillons a entraîné une baisse de 24 points de la moyenne et de 115 points de l'écart-type ([A1-4](micro-table-A1-4)).
:::

:::{card} 

{glue}`fig-mp3` 

+++
<b> Figure 1.3 :</b> Pla'stock 2022: distribution des résultats par lieu d'échantillonage.
:::


(conditions_mp)=
### Conditions d'échantillonage

::::{tab-set}

:::{tab-item} Resumé
:selected:

{glue}`table2`

{glue}`table3`


::: 

:::{tab-item} Fibres

{glue}`table4`

{glue}`table5`

{glue}`table6`

{glue}`table7`

:::

:::{tab-item} Fragments

{glue}`table8`

{glue}`table9`

{glue}`table10`

{glue}`table11`
::: 

:::{tab-item} Souples

{glue}`table12`

{glue}`table13`

{glue}`table14`

{glue}`table15`
:::

::::

### Régionalité

Définition des régions: [Léman régions](leman_regions).

::::{grid} 1 1 2 2

:::{grid-item}

{glue}`table16`

:::

:::{grid-item}

{glue}`table17`

:::

:::{grid-item}

{glue}`table18`

:::

:::{grid-item}



:::

::::


In [28]:
def analyze_scenario(scenario_data, func, n_iterations=100, bin_width=0.2):
    """
    Analyze a specific scenario using Random Forest regression with bootstrapping,
    and calculate feature importances.

    :param data: DataFrame containing the dataset.
    :param feature_1: The name of the first feature for filtering.
    :param feature_1_value: The value of the first feature to filter by.
    :param feature_2: The name of the second feature for filtering.
    :param feature_2_value: The value of the second feature to filter by.
    :param n_iterations: Number of bootstrap iterations. Default is 100.
    :param bin_width: Width of each bin for histogram. Default is 0.2.
    :return: A tuple containing bins, bin probabilities, flattened predictions, and feature importances.
    """
    
    # Prepare data for regression
    y_scaler = MinMaxScaler()
    y_scaled = y_scaler.fit_transform(scenario_data['pcs_m'].values.reshape(-1,1)).flatten()
    
    # Initialize the OneHotEncoder
    # here we encode the ordinal data
    encoder = OneHotEncoder(sparse_output=False)
    
    X = scenario_data.drop('pcs_m', axis=1)
    
    # Apply the encoder to the categorical columns
    encoded_data = encoder.fit_transform(scenario_data[['fréquentation', 'situation', 'distance', 'substrat']])
    # Create a DataFrame with the encoded data
    X_encoded = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['fréquentation', 'situation', 'distance', 'substrat']))

    
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_scaled, test_size=0.1, random_state=42)

    # Bootstrap predictions and accumulate feature importances
    bootstrap_predictions = []
        
    # Collect diagnostic at each repetition
    cum_mse = []
    cum_r2 = []
    
    for _ in range(n_iterations):
        X_train_sample, y_train_sample = resample(X_train, y_train, n_samples= len(y_train) -1)
        rf_model_sample = func
        rf_model_sample.fit(X_train_sample, y_train_sample)
        
        pred = rf_model_sample.predict(X_test)
        
        r2 = r2_score(y_test, pred)
        pred = y_scaler.inverse_transform(pred.reshape(-1, 1)).flatten()
        bootstrap_predictions.append(pred)
        mse = mean_squared_error(y_test , pred)
        
        cum_mse.append(mse)
        cum_r2.append(r2)           

    # Flatten the predictions array
    predictions_flat = np.array(bootstrap_predictions).flatten()
    
    return predictions_flat, cum_mse, cum_r2

def plot_histogram(predictions, observed, title="", reference='camp-dist-1', display=False, order='predictions', xmax=800, n_bins=20, x_label='pcs/m³'):
    
    fig, ax = plt.subplots(figsize=(10, 6))
    if order == 'predictions':
        sns.histplot(predictions, binwidth=n_bins, stat="probability", ax=ax, label='prédictions', zorder=0)
        sns.histplot(observed, binwidth=n_bins, stat="probability", label='observée', zorder=1, ax=ax)
    else:
        sns.histplot(predictions, binwidth=n_bins, stat="probability", ax=ax, label='prédictions', zorder=1)
        sns.histplot(observed, binwidth=n_bins, stat="probability", label='observée', zorder=0, ax=ax)
    ax.set_xlim(-.1, xmax)
    plt.title(title, loc='left')
    plt.xlabel(x_label)
    plt.ylabel('Densité de Probabilité')
    plt.legend()
    glue(reference, fig, display=display)
    plt.close()

def evalutate_model(r2s, mses, label, model='random-forest'):
    r2 = np.round(np.mean(r2s), 2)
    mse = np.round(np.mean(mses), 2)
    results = {"cross validated error":r2, "mean² error":mse, 'model':model}
    return pd.DataFrame(results, index=[label])

# Calculating quantiles for Scenario 2
format_kwargs = dict(precision=0, thousands="'")
q_uants = [0.01, 0.25, 0.5, 0.75, 0.99]
index = ['1%', '25%', '50%', '75%', '99%', 'Moyenne']
def makeqdf(observed, predicted, index=index, quants=q_uants, caption=""):
    
    o_q = np.quantile(observed, quants)
    m_o = np.mean(observed)
    o_p = np.quantile(predicted, quants)
    m_p = np.mean(predicted)
    
    results = {'observée':[*o_q, m_o], 'prédiction': [*o_p, m_p]}
   
    return pd.DataFrame(results, index=index).style.set_table_styles(table_css_styles_top).format(**format_kwargs).set_caption(caption)

cols = ['échantillon', 'position', 'fréquentation','situation', 'distance', 'substrat']

In [29]:
# model parameters
estimators = 10
iterations = 100

test_xt = work_data_combined[~work_data_combined["échantillon"].isin(not_these)].copy()
# the volume of the container used to sample in cm³
# sample_volume_cm = 10*10*5
# sample_volume_m = sample_volume_cm/10**6
test_xt['pcs_m'] = test_xt['compte']


# Filter for Scenario 
test_xi = test_xt[(test_xt['position'] == 1)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()

test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat',  'pcs_m']]

func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42, max_samples=.85)
predictions_flat, feature_importances,  mse, r2 = pstk.analyze_scenario(test_x, func,  n_iterations=iterations)

caption = 'Ligne d\'eau'
q_sit_2_freq_3 = pstk.makeqdf(test_x.pcs_m.values, predictions_flat, caption=caption)
glue('q-lignedeau-sa', q_sit_2_freq_3, display=False)

# the histogram for this scenario:
title = 'Pla\'stock 2022, Le Léman micro particules, Le Léman\nDistribution des Prédictions - Ligne d\'eau'
pstk.plot_histogram(predictions_flat, test_x.pcs_m.values, title=title, reference='lignedeau-sa', display=False)

In [30]:
# Filter for Scenario 

test_xt = work_data_combined[~work_data_combined["échantillon"].isin(not_these)].copy()
# the volume of the container used to sample in cm³
# sample_volume_cm = 10*10*5
# sample_volume_m = sample_volume_cm/10**6
test_xt['pcs_m'] = test_xt['compte']


# Filter for Scenario 
test_xi = test_xt.copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()

test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat',  'pcs_m']]

func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42, max_samples=.85)
predictions, feature_importances,  mse, r2 = pstk.analyze_scenario(test_x, func,  n_iterations=iterations)


caption = 'Tous les positions'
q_sit_2_freq_3 = pstk.makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-tous-md-sa', q_sit_2_freq_3, display=False)

# the histogram for this scenario:
title = 'Pla\'stock 2022, Le Léman micro particules, Le Léman\nDistribution des Prédictions'
pstk.plot_histogram(predictions_flat, test_x.pcs_m.values, title=title, reference='tous-md-sa', display=False)

In [31]:
# Filter for Scenario 
test_xi = test_xt[(test_xt['position'] == 2)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()

test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]

func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42, max_samples=.9, min_samples_leaf=2)
predictions, feature_importances,  mse, r2 = pstk.analyze_scenario(test_x, func,  n_iterations=iterations)

caption = 'Plage seche'
q_sit_2_freq_3 = pstk.makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-plageseche-sa', q_sit_2_freq_3, display=False)

# the histogram for this scenario:
title = 'Pla\'stock 2022 , Le Léman micro particules\nDistribution des Prédictions - Plage seche'
pstk.plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='plageseche-sa', display=False)

In [32]:
# Filter for Scenario 
cols = ['échantillon', 'position', 'fréquentation','situation', 'distance', 'substrat']
test_xi = test_xt[(test_xt['position'] == 2)&(test_xt.objet == 'fibres')].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()

test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]

func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42, max_samples=.85)
predictions, feature_importances,  mse, r2 = pstk.analyze_scenario(test_x, func,  n_iterations=iterations)

caption = 'Plage seche et fibres'
q_sit_2_freq_3 = pstk.makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-plagesechefibres-sa', q_sit_2_freq_3, display=False)

# the histogram for this scenario:
title = 'Pla\'stock 2022, Le Léman micro particules\nDistribution des Prédictions - Plage seche et fibres'
pstk.plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='plagesechefibres-sa', display=False)

In [33]:
# Filter for Scenario 
test_xi = test_xt[(test_xt['substrat'] == 1)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()

test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]

func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42, max_samples=.9, min_samples_leaf=3)
predictions, feature_importances,  mse, r2 = pstk.analyze_scenario(test_x, func,  n_iterations=iterations)

caption = 'Sable'
q_sit_2_freq_3 = pstk.makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-sablesfins-sa', q_sit_2_freq_3, display=False)

# the histogram for this scenario:
title = 'Pla\'stock 2022, Le Léman\nDistribution des Prédictions - Sables fins'
pstk.plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='sablesfins-sa', display=False)

In [34]:
# Filter for Scenario 
test_xi = test_xt[(test_xt['substrat'] == 2)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()

test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]

func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42, max_samples=.85)
predictions, feature_importances,  mse, r2 = pstk.analyze_scenario(test_x, func,  n_iterations=iterations)

caption = 'Graviers'
q_sit_2_freq_3 = pstk.makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-cailloux-sa', q_sit_2_freq_3, display=False)

# the histogram for this scenario:
title = 'Pla\'stock 2022, Le Léman micro particules\nDistribution des Prédictions - Gravier'
pstk.plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='cailloux-sa', display=False)

In [35]:
# Filter for Scenario 
test_xi = test_xt[(test_xt['fréquentation'] == 3)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()

test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]

func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42, max_samples=.3)
predictions, feature_importances,  mse, r2 = pstk.analyze_scenario(test_x, func,  n_iterations=iterations)

caption = 'Fréquentation élevée'
q_sit_2_freq_3 = pstk.makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-freq3-sa', q_sit_2_freq_3, display=False)

# the histogram for this scenario:
title = 'Pla\'stock 2022, Le Léman micro particules\nDistribution des Prédictions - Fréquentation élevée'
pstk.plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='freq3-sa', display=False)

In [36]:
# Filter for Scenario 
test_xi = test_xt[(test_xt['fréquentation'] == 2)].copy()
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()

test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]

func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42, max_samples=.85)
predictions, feature_importances,  mse, r2  = pstk.analyze_scenario(test_x, func,  n_iterations=iterations)

caption = 'Fréquentation moyenne'
q_sit_2_freq_3 = pstk.makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-freq2-sa', q_sit_2_freq_3, display=False)

# the histogram for this scenario:
title = 'Pla\'stock 2022, Le Léman micro particules\nDistribution des Prédictions - Fréquentation moyenne'
pstk.plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='freq2-sa', display=False)

(random_forest_sa_md)=
### Random Forest 

Source : [scikit-learn random forest](https://scikit-learn.org/0.16/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

criterion : `absolute error`

La régression avec forêt aléatoire est une technique d'apprentissage automatique (machine learning) utilisée pour prédire des résultats continus (par opposition aux catégories dans la classification). C'est une méthode d'apprentissage ensembliste, ce qui signifie qu'elle combine les prédictions de plusieurs algorithmes d'apprentissage automatique pour produire des prédictions plus précises.

::::{tab-set}

:::{tab-item} Toutes les conditions
{glue}`tous-md-sa`
:::

:::{tab-item} Fréquentation moyenne
{glue}`freq2-sa`
:::

:::{tab-item} Haute fréquentation
{glue}`freq3-sa`
:::

:::{tab-item} Graviers
{glue}`cailloux-sa`
:::

:::{tab-item} Sable
{glue}`sablesfins-sa`
:::

:::{tab-item} Plage seche et sable fins
{glue}`plagesechefibres-sa`

:::

:::{tab-item} Plage seche
{glue}`plageseche-sa`

:::

:::{tab-item} Ligne d'eau
{glue}`lignedeau-sa`

:::


:::{tab-item} Résultats
:selected:

````{grid} 1 2 2 2

```{grid-item}
{glue}`q-tous-md-sa`
```

```{grid-item}
{glue}`q-lignedeau-sa`

```

```{grid-item}
{glue}`q-freq2-sa`
```

```{grid-item}
{glue}`q-freq3-sa`
```

```{grid-item}
{glue}`q-cailloux-sa`
```

```{grid-item}
{glue}`q-sablesfins-sa`
```

```{grid-item}
{glue}`q-plagesechefibres-sa`
```

```{grid-item}
{glue}`q-plageseche-sa`
```

```{grid-item}

```

````
:::

::::    

In [37]:
col = 'top'
pcs_col = 'particules'
cols = ['échantillon', 'fréquentation', 'situation', 'distance', 'substrat']
grid_range = np.arange(0, 1001)

# The data to analyze and a beta prior
test_xt = work_data_combined[~work_data_combined["échantillon"].isin(not_these)].copy()
beta_prior = pstk.calculate_beta_prior(grid_range=grid_range, bin_density_numbers=[1])

bins = [1]

test_x = test_xt.groupby(cols, as_index=False)[pcs_col].sum()
test_x['top'] = 1

grid_pstock = pstk.calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock1 = pstk.define_posterior(grid_pstock.copy(), beta_prior, grid_val_index=grid_range)

# samples
sample_totals = pstk.posterior_predictions(post_grid_pstock1)

caption = 'Toutes les conditions'

test_grid_quants = pstk.makeqdf(test_x[pcs_col].values, sample_totals, caption=caption)
glue('q-micro-tous', test_grid_quants, display=False)

title = 'Pla\'stock 2022, Le Léman micro particules,\nDistribution des Prédictions: toutes les conditions,  approximation Bayésienne, prior = beta(1,1)'

pstk.plot_histogram(sample_totals, test_x[pcs_col].values, title=title, reference='micro-toutes-ga', display=False, xlabel=pcs_col)

In [38]:
col = 'fréquentation'
pcs_col = 'particules'
cols = ['échantillon', 'fréquentation', 'situation', 'distance', 'substrat']
bins = [3]

# beta_prior = pstk.calculate_beta_prior(grid_range=grid_range, bin_density_numbers=[1])

test_x = test_xt[(test_xt['fréquentation'] == 3)].copy()
test_x = test_x.groupby(cols, as_index=False)[pcs_col].sum()


grid_pstock = pstk.calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock2 = pstk.define_posterior(grid_pstock.copy(), beta_prior, grid_val_index=grid_range)

# samples
sample_totals = pstk.posterior_predictions(post_grid_pstock2)

caption = 'Fréquentation élevée'

test_grid_quants = pstk.makeqdf(test_x[pcs_col].values, sample_totals, caption=caption)
glue('q-micro-freqelevee', test_grid_quants, display=False)

title = 'Pla\'stock 2022, Le Léman micro particules\nDistribution des Prédictions: Fréquentation élevée,  approximation Bayésienne, prior = beta(1,1)'

pstk.plot_histogram(sample_totals, test_x[pcs_col].values, title=title, reference='micro-freqelevee-ga', display=False)

In [39]:
col = 'fréquentation'
pcs_col = 'particules'
cols = ['échantillon', 'fréquentation', 'situation', 'distance', 'substrat']
bins = [2]

# beta_prior = pstk.calculate_beta_prior(grid_range=grid_range, bin_density_numbers=[1])

test_x = test_xt[(test_xt['fréquentation'] == 2)].copy()
test_x = test_x.groupby(cols, as_index=False)[pcs_col].sum()


grid_pstock = pstk.calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock2 = pstk.define_posterior(grid_pstock.copy(), beta_prior, grid_val_index=grid_range)

# samples
sample_totals = pstk.posterior_predictions(post_grid_pstock2)

caption = 'Fréquentation moyenne'

test_grid_quants = pstk.makeqdf(test_x[pcs_col].values, sample_totals, caption=caption)
glue('q-micro-freqmoyenne', test_grid_quants, display=False)

title = 'Pla\'stock 2022, Le Léman micro particules\nDistribution des Prédictions: Fréquentation moyenne,  approximation Bayésienne, prior = beta(1,1)'

pstk.plot_histogram(sample_totals, test_x[pcs_col].values, title=title, reference='micro-freqmoyenne-ga', display=False)

In [40]:
col = 'substrat'
pcs_col = 'particules'
# grid_range = grid_val_index
# category value
cat = 1
bins = [cat]
# test_xt = work_data_combined[~work_data_combined["échantillon"].isin(not_these)].copy()
# test_xt['pcs_m'] = test_xt['compte']

mask = (test_xt[col] == cat)&(test_xt.substrat == cat)
test_x = test_xt[mask].copy()

test_x = test_x.groupby(cols, as_index=False)[pcs_col].sum()

grid_pstock = pstk.calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock3 = pstk.define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# samples
sample_totals = pstk.posterior_predictions(post_grid_pstock3.copy())

caption = 'Sable'

test_grid_quants = pstk.makeqdf(test_x[pcs_col].values, sample_totals, caption=caption)
glue('q-micro-sable', test_grid_quants, display=False)
title = 'Pla\'stock 2022, Le Léman micro particules\nDistribution des Prédictions: Sable, approximation Bayésienne, prior = beta(1,1)'

pstk.plot_histogram(sample_totals, test_x[pcs_col].values, title=title, reference='micro-sable-ga', display=False)

In [41]:
col = 'substrat'
pcs_col = 'particules'
# grid_range = grid_val_index
# category value
cat = 2
bins = [cat]
# test_xt = work_data_combined[~work_data_combined["échantillon"].isin(not_these)].copy()
# test_xt['pcs_m'] = test_xt['compte']

mask = (test_xt[col] == cat)
test_x = test_xt[mask].copy()



test_x = test_x.groupby(cols, as_index=False)[pcs_col].sum()

grid_pstock = pstk.calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock3 = pstk.define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# samples
sample_totals = pstk.posterior_predictions(post_grid_pstock3.copy())

caption = 'Graviers'

test_grid_quants = pstk.makeqdf(test_x[pcs_col].values, sample_totals, caption=caption)
glue('q-micro-graviers', test_grid_quants, display=False)
title = 'Pla\'stock 2022, Le Léman micro particules\nDistribution des Prédictions: Graviers,  approximation Bayésienne, prior = beta(1,1)'

pstk.plot_histogram(sample_totals, test_x[pcs_col].values, title=title, reference='micro-graviers-ga', display=False)

In [42]:
mask = (test_xt.position == 2)&(test_xt.objet == 'fibres')
test_x = test_xt[mask].copy()

test_x = test_x.groupby(cols, as_index=False)[pcs_col].sum()

grid_pstock = pstk.calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock3 = pstk.define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# samples
sample_totals = pstk.posterior_predictions(post_grid_pstock3.copy())

caption = 'Plage seche et fibres'

test_grid_quants = pstk.makeqdf(test_x[pcs_col].values, sample_totals, caption=caption)
glue('q-micro-sechefibres', test_grid_quants, display=False)
title = 'Pla\'stock 2022, Le Léman micro particules\nDistribution des Prédictions: Plage seche et fibres, approximation Bayésienne, prior = beta(1,1)'

pstk.plot_histogram(sample_totals, test_x[pcs_col].values, title=title, reference='micro-sechefibres-ga', display=False)

In [43]:
mask = (test_xt.position == 2)
test_x = test_xt[mask].copy()



test_x = test_x.groupby(cols, as_index=False)[pcs_col].sum()

grid_pstock = pstk.calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock3 = pstk.define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# samples
sample_totals = pstk.posterior_predictions(post_grid_pstock3.copy())

caption = 'Plage seche'

test_grid_quants = pstk.makeqdf(test_x[pcs_col].values, sample_totals, caption=caption)
glue('q-micro-seche', test_grid_quants, display=False)
title = 'Pla\'stock 2022, Le Léman micro particules\nDistribution des Prédictions: Plage seche,  approximation Bayésienne, prior = beta(1,1)'

pstk.plot_histogram(sample_totals, test_x[pcs_col].values, title=title, reference='micro-seche-ga', display=False)

In [44]:
mask = (test_xt.position == 1)
test_x = test_xt[mask].copy()



test_x = test_x.groupby(cols, as_index=False)[pcs_col].sum()

grid_pstock = pstk.calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock3 = pstk.define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# samples
sample_totals = pstk.posterior_predictions(post_grid_pstock3.copy())

caption = 'Ligne d\'eau'

test_grid_quants = pstk.makeqdf(test_x[pcs_col].values, sample_totals, caption=caption)
glue('q-micro-lignedeau-ga', test_grid_quants, display=False)
title = 'Pla\'stock 2022, Le Léman micro particules\nDistribution des Prédictions: Ligne d\'eau, approximation Bayésienne, prior = beta(1,1)'

pstk.plot_histogram(sample_totals, test_x[pcs_col].values, title=title, reference='micro-lignedeau-ga', display=False)

(grid_approx_micro)=
### Approximation Bayésienne par Grille

Source : [solid-waste-team](https://hammerdirt-analyst.github.io/solid-waste-team/grid_approximation.html)

prior : beta(1,1)

Cas d'utilisation : Cette méthode est une approche manuelle de l'inférence Bayésienne. Elle est particulièrement utile lorsque vous souhaitez incorporer des croyances antérieures et mettre à jour ces croyances avec des données observées.

Mise en œuvre : Implique la définition d'une grille de valeurs de paramètres et le calcul de la vraisemblance des données observées à chaque point de cette grille. En multipliant par la probabilité a priori et en normalisant, on obtient la distribution a posteriori. Cela peut être fait pour chaque condition séparément ou pour toutes les conditions ensemble, bien que cela soit plus intensif en termes de calcul.



::::{tab-set}

:::{tab-item} Toutes les conditions
{glue}`micro-toutes-ga`
:::

:::{tab-item} Fréquentation moyenne
{glue}`micro-freqmoyenne-ga`
:::

:::{tab-item} Haute fréquentation
{glue}`micro-freqelevee-ga`
:::

:::{tab-item} Graviers
{glue}`micro-graviers-ga`
:::

:::{tab-item} Sable
{glue}`micro-sable-ga`
:::

:::{tab-item} Plage seche et sable fins
{glue}`micro-sechefibres-ga`

:::

:::{tab-item} Plage seche
{glue}`micro-seche-ga`

:::

:::{tab-item} Ligne d'eau
{glue}`micro-lignedeau-ga`

:::


:::{tab-item} Résultats
:selected:

````{grid} 1 2 2 2

```{grid-item}
{glue}`q-micro-tous`
```

```{grid-item}

{glue}`q-micro-lignedeau-ga`

```

```{grid-item}
{glue}`q-micro-freqmoyenne`
```

```{grid-item}
{glue}`q-micro-freqelevee`
```

```{grid-item}
{glue}`q-micro-graviers`
```

```{grid-item}
{glue}`q-micro-sable`
```

```{grid-item}
{glue}`q-micro-sechefibres`
```

```{grid-item}
{glue}`q-micro-seche`
```

```{grid-item}

```

````
:::

::::    

In [45]:
%watermark --iversions -b -r

Git repo: https://github.com/hammerdirt-analyst/plastock.git

Git branch: feb22

pandas    : 2.0.3
numpy     : 1.26.3
matplotlib: 3.8.2
seaborn   : 0.13.1

